In [ ]:
#历史数据-实时数据-开发调试-策略回测-模拟交易-实盘交易-运行监控-风险管理

In [ ]:
##导入模块 ###########################

import os
import time
import sqlite3 as sl

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import akshare as ak
import talib

In [ ]:
##多股票 参数 批量 ########

# coding: utf-8

# 1 import
import os
import time
import math
import sqlite3 as sl 

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import akshare as ak
import talib

# 历史数据-实时数据-开发调试-策略回测-模拟交易-实盘交易-运行监控-风险管理
# 2 define

timeStamp = int(round(time.time() * 1000))  # 1645371358737 #13位 毫秒
# '2017-11-07 16:47:14' %Y-%m-%d %H:%M:%S %Y%m%d
dateTime = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
today = time.strftime('%Y%m%d')  # 20220221 #8位
startTime = time.time()

dirPath = '/home/at/ipynb/QL'
curDir = os.getcwd()  # 获取当前工作目录路径

eNameFile = 'eName.csv'
eNamePath = os.path.join(dirPath, eNameFile)
eName_df = pd.read_csv(eNamePath)

xgFile = 'XG.EBK'   # XG.blk XG.EBK
xgPath = os.path.join(dirPath, xgFile)

maTacFile = 'maTac.csv'
maTaccsvPath = os.path.join(dirPath, maTacFile)

maTacticFile = 'maTactic.pkl'
maTacticPath = os.path.join(dirPath, maTacticFile)
# maTacticdf = pd.read_pickle(maTacticPath)
maTacticdf = pd.DataFrame(columns=['SecurityID', 'rUpDown', 'Date', 'winRate', 'tradeCount',
                'winLim', 'winCount', 'perWin', 'lossLim', 'lossCount', 'perLoss',
                'dayLim', 'OverCount', 'perOver', 'cDownLim', 'cDownCount', 'percDown'])
# print(f'{timeStamp = } , {dateTime = } , {today = } ')

tradeFile = 'trade.pkl'
tradePath = os.path.join(dirPath, tradeFile)
# tradedf = pd.read_pickle(tradePath)
tradedf = pd.DataFrame(columns=['rindex','SecurityID', 'TradeDate', 'Close', 'High', 'Low', 'winLim',
            'lossLim','dayLim','cDownLim','rUpDown', 'posiDay', 'signal', 'buyPrice', 'sellPrice', 
            'curRate','winRate', 'tradeCount', 'winCount', 'lossCount', 'OverCount', 'cDownCount'])

kHistFile = 'kHist.pkl'
kHistPath = os.path.join(dirPath, kHistFile)
kHistdf = pd.read_pickle(kHistPath)

# 3 akshare spotStock ST stop
# stock_zh_a_stop_em_df = ak.stock_zh_a_stop_em() #退市
stock_zh_a_st_em_df = ak.stock_zh_a_st_em()  # ST股
eName_st_df = eName_df[eName_df.dfcf.isin(stock_zh_a_st_em_df.columns.values.tolist())]
eName_st_df = eName_df[['dfcf', 'En']].reset_index(drop=True)
st_df = stock_zh_a_st_em_df.copy(deep=True)
st_df.rename(columns=dict(eName_st_df.values.tolist()), inplace=True)

stock_tfp_em_df = ak.stock_tfp_em(date=today)  # 停复牌 20220221
eName_stop_df = eName_df[eName_df.dfcf.isin(stock_tfp_em_df.columns.values.tolist())]
eName_stop_df = eName_df[['dfcf', 'En']].reset_index(drop=True)
stop_df = stock_tfp_em_df.copy(deep=True)
stop_df.rename(columns=dict(eName_stop_df.values.tolist()), inplace=True)

# time.sleep(1)  # 休眠1秒

stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()

eName_aSpot = eName_df[eName_df.dfcf.isin(stock_zh_a_spot_em_df.columns.values.tolist())]
eName_aSpot = eName_aSpot[['dfcf', 'En']].reset_index(drop=True)
stock_zh_a_spot_em_df.rename(columns=dict(eName_aSpot.values.tolist()), inplace=True)

aSpot = stock_zh_a_spot_em_df.copy(deep=True)
aSpot.drop(aSpot.index, inplace=True)  # 表头 空表

# time.sleep(1)  # 休眠1秒

# 4 block load
with open(xgPath, 'r', encoding='utf-8') as f:
    str_data = f.read().splitlines()

print('str_data len:', len(str_data)-1, '\t', time.strftime("%Y-%m-%d %H:%M:%S"), '\n', str_data)


stStock = []
stDict = {}
stopStock = []
stopDict = {}

for iXg in str_data:
    SecurityID = iXg[-6:]

    if SecurityID in st_df.SecurityID.values.tolist():
        stStock.append(SecurityID)
        stDict[SecurityID] = st_df.Symbol.loc[st_df["SecurityID"] == SecurityID].values[0]

    if SecurityID in stop_df.SecurityID.values.tolist():
        stopStock.append(SecurityID)
        stopDict[SecurityID] = stop_df.Symbol.loc[stop_df["SecurityID"] == SecurityID].values[0]

    if len(SecurityID) == 6:
        aSpot = aSpot.append(
            stock_zh_a_spot_em_df.loc[stock_zh_a_spot_em_df["SecurityID"] == SecurityID])
    else:
        continue

    if stDict:
        print(f'{stDict = }')

    if stopDict:
        print(f'{stopDict = }', '\n')

aSpot = aSpot.reset_index(drop=True)

for indexA, rowA in aSpot.iterrows():  # 品种循环
    SecurityID = rowA['SecurityID']
    if len(SecurityID) != 6:
        continue

    # stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=SecurityID, period="daily", adjust="qfq")
    # kLinedf = stock_zh_a_hist_df.tail(700).copy(deep=True)  # 140 280 420 560 700

    kLinedf = kHistdf[kHistdf['代码'] == SecurityID].tail(700).copy(deep=True)  # 140 280 420 560 700

    kLinedf = kLinedf.reset_index(drop=True)
    eName_hist = eName_df[eName_df.dfcf.isin(kLinedf.columns.values.tolist())]
    eName_hist = eName_hist[['dfcf', 'En']].reset_index(drop=True)
    kLinedf.rename(columns=dict(eName_hist.values.tolist()), inplace=True)

    for iList in list(range(2, 20+1)) + [30, 60, 90, 120, 250]:
        kLinedf['Ma'+str(iList)] = talib.MA(kLinedf.Close.values,timeperiod=iList)  # 多均线

    rowCount = 0
    sumClose = 0.0
    for rindex, row in kLinedf.iterrows():
        rowCount += 1
        sumClose += row['Close']
        maClose = round(sumClose/rowCount, 3)

        for iList in list(range(2, 20+1)) + [30, 60, 90, 120, 250]:
            Ma = 'Ma'+str(iList)
            if math.isnan(row[Ma]):
                kLinedf.at[rindex, Ma] = maClose  # 移动均线 nan 补均值
            else:
                kLinedf.at[rindex, Ma] = round(row[Ma], 3)

    for i in range(2, 20+1):
        Ma = 'Ma'+str(i)
        cUp = 'cUp'+str(i)
        kLinedf[cUp] = 0
        kLinedf.loc[kLinedf.Close[(kLinedf.Close > kLinedf[Ma]) & (
            kLinedf.Close.shift() < kLinedf[Ma].shift())].index, cUp] = 1  # 金叉

        cDown = 'cDown'+str(i)
        kLinedf[cDown] = 0
        kLinedf.loc[kLinedf.Close[(kLinedf.Close < kLinedf[Ma]) & (
            kLinedf.Close.shift() > kLinedf[Ma].shift())].index, cDown] = -1  # 死叉

        # kLinedf['cUp'+str(i)] = kLinedf[['Close',Ma]].apply(lambda row: True if row.Close > row[Ma]
        #     and row.Close.shift(periods=-1, axis=0) < row[Ma].shift(periods=-1, axis=0)
        #     else False, axis=1)

    for i in [30, 60, 90, 120, 250]:
        Ma = 'Ma'+str(i)
        kLinedf['On'+str(i)] = 0
        kLinedf.loc[kLinedf.Close[(
            kLinedf.Close > kLinedf[Ma])].index, 'On'+str(i)] = 1  # 均线之上

    # winLim = 0.03  # 止盈
    # lossLim = -0.05  # 止损
    # dayLim = 5  # 持仓天数限制
    # cDownLim = -1  # 死叉信号

    freeRate = 0.997  # 税费率
    for i in range(2, 20+1):  # 品种->均线1金叉->循环
        cUp = 'cUp'+str(i)

        for cDownLim in [-1, -2]:  # 死叉信号 -2禁用

            for j in range(2, 20+1):  # 品种->均线1金叉->均线2死叉->循环
                tUpDown = 'tU'+str(i) + 'D'+str(j)  # 交易信号 数组列
                kLinedf[tUpDown] = 0

                # bUpDown = 'bU'+str(i) +'D'+str(j)
                # kLinedf[bUpDown] = kLinedf.Close

                # sUpDown = 'sU'+str(i) +'D'+str(j)
                # kLinedf[sUpDown] = kLinedf.Close

                rUpDown = 'rU'+str(i) + 'D'+str(j)  # 收益率 数组列
                kLinedf[rUpDown] = 1.0

                cDown = 'cDown'+str(j)  # 死叉信号 数组列

                kLinedf.loc[(kLinedf[cUp] == 1) & ((kLinedf['On60'] == 1) | (kLinedf['On90'] == 1) | (
                    kLinedf['On120'] == 1) | (kLinedf['On250'] == 1)), tUpDown] = 1  # 交易信号 数组列 金叉+均线之上

                for winLim in [0.03, 0.05]:  # 止盈
                    for lossLim in [-0.03, -0.05, -2]:  # 止损 -2禁用
                        for dayLim in [3, 5, 7, 10]:  # 持仓天数限制

                            ####################################
                            # winLim = 0.03  # 止盈
                            # lossLim = -0.05  # 止损
                            # dayLim = 5  # 持仓天数限制
                            # cDownLim = -1  # 死叉信号

                            signal = 0    # 当前交易信号
                            winRate = 1.0  # 总收益率 *
                            posiDay = 0   # 持仓天数
                            posiVol = 0   # 仓位 股数
                            initAmt = 0   # 初始金额
                            buyPrice = 0.0  # 买价
                            sellPrice = 0.0  # 卖价

                            tradeCount = 0  # 总交易次数 单品种 累计
                            winCount = 0  # 止盈平仓次数
                            lossCount = 0  # 止损平仓次数
                            OverCount = 0  # 超期平仓次数
                            cDownCount = 0  # 死叉平仓次数

                            kLinedf.loc[(kLinedf[cUp] == 1) & ((kLinedf['On60'] == 1) | (kLinedf['On90'] == 1) | (
                                kLinedf['On120'] == 1) | (kLinedf['On250'] == 1)), tUpDown] = 1  # 交易信号 数组列 金叉+均线之上

                            for rindex, row in kLinedf.iterrows():  # 品种->均线1金叉->均线2死叉->行->循环
                                signal = row[tUpDown]
                                if posiDay > 0:  # 持仓
                                    kLinedf.at[rindex, rUpDown] = winRate
                                    posiDay += 1
                                    if row[tUpDown] == 1:  # 屏蔽持仓期的信号
                                        signal = 0
                                        kLinedf.at[rindex, tUpDown] = signal

                                    # 止盈平仓
                                    if row["High"] > buyPrice * (1 + winLim):
                                        signal = -1
                                        posiDay = 0
                                        sellPrice = round(
                                            buyPrice * (1 + winLim), 2)
                                        winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
                                        kLinedf.at[rindex, rUpDown] = winRate
                                        winCount += 1
                                        # print(rUpDown, rindex, signal, buyPrice, sellPrice, winRate)

                                    # 止损平仓
                                    elif row["Low"] < buyPrice * (1 + lossLim):
                                        signal = -2
                                        posiDay = 0
                                        sellPrice = round( buyPrice * (1 + lossLim), 2)
                                        winRate = round( winRate * sellPrice / buyPrice * freeRate, 3)
                                        kLinedf.at[rindex, rUpDown] = winRate
                                        lossCount += 1
                                        # print(rUpDown, rindex, signal, buyPrice, sellPrice, winRate)

                                    elif row[cDown] == cDownLim:  # 死叉平仓
                                        signal = -4
                                        posiDay = 0
                                        sellPrice = row["Close"]
                                        winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
                                        kLinedf.at[rindex, rUpDown] = winRate
                                        cDownCount += 1

                                    elif posiDay >= dayLim:  # 超期平仓
                                        signal = -3
                                        posiDay = 0
                                        sellPrice = row["Close"]
                                        winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
                                        kLinedf.at[rindex, rUpDown] = winRate
                                        OverCount += 1
                                        # print(rUpDown, rindex, signal, buyPrice, sellPrice, winRate)

                                    kLinedf.at[rindex, tUpDown] = signal

                                elif posiDay == 0:
                                    if row[tUpDown] == 1:  # 交易信号，开仓 买入价
                                        signal = 1
                                        posiDay = 1
                                        buyPrice = row["Close"]
                                        tradeCount += 1

                                else:
                                    pass

                                kLinedf.at[rindex, rUpDown] = winRate
                            ####################################

                            perWin = round(winCount/tradeCount * 100, 1)  # 止盈平仓次数占比%
                            perLoss = round(lossCount/tradeCount*100, 1)  # 止盈平仓次数占比%
                            perOver = round(OverCount/tradeCount*100, 1)  # 超期平仓次数占比%
                            percDown = round(cDownCount/tradeCount*100, 1)  # 死叉平仓次数占比%

                            # print(f"{SecurityID}:{rUpDown}:{winRate}:交易次数:{tradeCount}, {winLim}止盈:{winCount}次{perWin}%,
                            # {lossLim}止损:{lossCount}次{perLoss}%, {dayLim}日超期:{OverCount}次{perOver}%,
                            # {cDownLim}死叉:{cDownCount}次{percDown}%  ")

                            maTacticdf.loc[maTacticdf.shape[0]] = [SecurityID, rUpDown, today, winRate, tradeCount,
                                            winLim, winCount, perWin, lossLim, lossCount, perLoss, dayLim, OverCount,
                                            perOver, cDownLim, cDownCount, percDown]

                            ####################################

                if cDownLim == -2:
                    break

    # time.sleep(1)
    maTacticdf.to_pickle(maTacticPath)

    print(aSpot.shape[0]-1, '-', indexA, '', f'{SecurityID}', ':', time.time() - startTime, '\t', time.strftime("%Y-%m-%d %H:%M:%S"))

# 降序 False ,  'winRate', 'perWin' ,
maTacticdf = maTacticdf.sort_values(['winRate'], ascending=[False])
# print(maTacticdf.describe())

maTacticdf.to_pickle(maTacticPath)
maTacticdf.to_csv(maTaccsvPath, index=False)

print("Time used:", time.time() - startTime, '\t', time.strftime("%Y-%m-%d %H:%M:%S"), '\t rows:', maTacticdf.shape[0])

print(maTacticdf.describe())

exit


In [ ]:
##结果 分析####################

maTac = pd.read_pickle('/home/at/ipynb/QL/maTactic.pkl')
# maTac2 = pd.read_pickle('/home/at/ipynb/QL/maTactic_20220317B.pkl')
# maTac= maTac.append(maTac2)
# print(maTac.describe())
maTac = maTac.sort_values(['winRate'], ascending=[False])
# print(maTac.describe())

# maTac.to_pickle('/home/at/ipynb/QL/maTactic_20220317.pkl')
# maTac.to_csv('/home/at/ipynb/QL/maTactic_20220317.csv', index=False)

winMax = maTac.groupby(['SecurityID'], as_index=False)['winRate'].max()
print( winMax )
# print(winMax.values.tolist() )

maTac_winMax = maTac.copy(deep=True)
maTac_winMax.drop(maTac_winMax.index, inplace=True)  # 表头 空表

for rindex, row in winMax.iterrows():
    tmp = maTac.loc[(maTac.SecurityID == row.SecurityID) & (maTac.winRate == row.winRate)]
    # print(tmp.describe())
    # print(tmp[0:])
    maTac_winMax = maTac_winMax.append(tmp[0:1], ignore_index=True)

maTac_winMax = maTac_winMax.sort_values(['winRate'], ascending=[False])
print(maTac_winMax)



  SecurityID  winRate
0     300390    3.732
  SecurityID rUpDown      Date  winRate tradeCount  winLim winCount  perWin  \
0     300390   rU2D2  20220321    3.732         74    0.05       58    78.4   

   lossLim lossCount  perLoss dayLim OverCount  perOver cDownLim cDownCount  \
0     -2.0         0      0.0     10        15     20.3       -2          0   

   percDown  
0       0.0  


In [ ]:
sz300390 = maTac.loc[(maTac.SecurityID == '300390') & (maTac.winLim  == 0.05) & (maTac.lossLim  == -0.03) & (maTac.dayLim  == 7)]
sz300390 = sz300390.sort_values(['winRate'], ascending=[False])
sz300390 = sz300390.reset_index(drop=True)
# print( sz300390.describe() )
print(sz300390[0:3])

In [ ]:
##导入模块 初始化 清空###########################

# import os
# import time
# import math

# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

# import akshare as ak
# import talib

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

today = time.strftime('%Y%m%d')  # 20220221 #8位
dirPath = '/home/at/ipynb/QL'
eNameFile = 'eName.csv'
eNamePath = os.path.join(dirPath, eNameFile)
eName_df = pd.read_csv(eNamePath)

xgFile = 'XG.EBK'   # XG.blk XG.EBK
xgPath = os.path.join(dirPath, xgFile)

maTacFile = 'maTac.csv'
maTaccsvPath = os.path.join(dirPath, maTacFile)

maTacticFile = 'maTactic.pkl'
maTacticPath = os.path.join(dirPath, maTacticFile)
# maTacticdf = pd.read_pickle(maTacticPath)
maTacticdf = pd.DataFrame(columns=['SecurityID', 'rUpDown', 'Date', 'winRate', 'tradeCount',
            'winLim', 'winCount', 'perWin', 'lossLim', 'lossCount', 'perLoss',
            'dayLim', 'OverCount', 'perOver', 'cDownLim', 'cDownCount', 'percDown'])

tradeFile = 'trade.pkl'
tradePath = os.path.join(dirPath, tradeFile)
# tradedf = pd.read_pickle(tradePath)
tradedf = pd.DataFrame(columns=['rindex','SecurityID', 'TradeDate', 'Close', 'High', 'Low', 'winLim',
            'lossLim','dayLim','cDownLim','rUpDown', 'posiDay', 'signal', 'buyPrice', 'sellPrice', 
            'curRate','winRate', 'tradeCount', 'winCount', 'lossCount', 'OverCount', 'cDownCount'])

kHistFile = 'kHist.pkl'
kHistPath = os.path.join(dirPath, kHistFile)
kHistdf = pd.read_pickle(kHistPath)
# kHistdf.to_pickle(kHistPath)


2022-03-23 18:52:25


In [ ]:
##多股票 获取历史数据 kHist########

loopCount = 0
with open(xgPath, 'r', encoding='utf-8') as f:
    str_data = f.read().splitlines()

for iXg in str_data:
    SecurityID = iXg[-6:]

    if len(SecurityID) == 6:
        # SecurityID = '300390'
        stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=SecurityID, period="daily", adjust="qfq")

        if len(stock_zh_a_hist_df.index) > 0:

            # eName_hist = eName_df[eName_df.dfcf.isin(stock_zh_a_hist_df.columns.values.tolist())]
            # eName_hist = eName_hist[['dfcf', 'En']].reset_index(drop=True)
            # stock_zh_a_hist_df.rename(columns=dict(eName_hist.values.tolist()), inplace=True)
            
            stock_zh_a_hist_df.insert(0, '代码', SecurityID, allow_duplicates=False)

            if 'kHistdf' not in dir():
                kHistdf = stock_zh_a_hist_df.copy(deep=True)
            else:
                kHistdf = kHistdf[~(kHistdf['代码'] == SecurityID)]
                kHistdf = kHistdf.append(stock_zh_a_hist_df, ignore_index=True)

                # kHistdf = kHistdf.reset_index(drop=True)
                # kHistdf.to_pickle(kHistPath)
                
            loopCount += 1
            curTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            print(f'[{curTime}-{len(str_data)-1}-{loopCount}-] :{SecurityID}')
            
            time.sleep(2)   #41xg 140s 

kHistdf.to_pickle(kHistPath)


In [141]:
##多股票 入库 历史数据 khist.db########
# import sqlite3 as sl

kHistdb = kHistdf.copy(deep=True)
eName_hist = eName_df[eName_df.dfcf.isin(kHistdb.columns.values.tolist())]
eName_hist = eName_hist[['dfcf', 'En']].reset_index(drop=True)
kHistdb.rename(columns=dict(eName_hist.values.tolist()), inplace=True)

conn = sl.connect('stock.db',check_same_thread=False, isolation_level=None, timeout=1000)
# conn.row_factory = sl.Row

sql = "select * from khist"

'''
cur = conn.cursor() # 游标
data = cur.execute(sql)

columns_tuple = cur.description # PRAGMA table_info(tab)
columns_list = [field_tuple[0] for field_tuple in columns_tuple] #列名 数组['ma', 'id']
query_result = cur.fetchall() #查询结果 数组[(1,'b'),()]

print(columns_list)

for row in cur:
    assert row[0] # == row["user_id"]
    # assert row["user_name"] == row["uSer_Name"]
    print(row[2]) # == row["user_pass"]

# print(cur.fetchone())  # ('1234', 'czx', '5678')
# print(cur.fetchmany())  #[('1234', 'czx', '5678')]
# print(cur.fetchall())  # [('1234', 'czx', '5678'), ('1235', 'ccc', '6666')]
# conn.commit()
'''

# sql_df = pd.read_sql(sql, conn) # pd读取sql
kHistdb.to_sql('khist', conn)  # pd写入sql


# if cur is not None:
#     cur.close()
if conn is not None:
    conn.close()


# 读取表
# with conn: 
#     data = conn.execute(sql) 
#     for row in data: 
#         print(row) 

# sqlite3 stock.db  .header on  .mode column  where rownum <=10
# alter table "kHistdf.db" rename to "khist" ;


In [ ]:
##单股票 参数########
SecurityID = '300390'

winLim = 0.233  # 止盈 0.03, 0.05
lossLim = -0.08  # 止损 -0.05 -0.03,-2
dayLim = 5  # 持仓天数限制 3, 5, 7, 10
cDownLim = -2  # 死叉信号  -1, -2 
freeRate = 0.997  # 税费率

signal = 0    # 当前交易信号
winRate = 1.0  # 总收益率 *
curRate = 0.0  # 当前收益率 *
posiDay = 0   # 持仓天数
posiVol = 0   # 仓位 股数
initAmt = 0   # 初始金额
buyPrice = 0.0  # 买价
sellPrice = 0.0  # 卖价

tradeCount = 0  # 总交易次数 单品种 累计
winCount = 0  # 止盈平仓次数
lossCount = 0  # 止损平仓次数
OverCount = 0  # 超期平仓次数
cDownCount = 0  # 死叉平仓次数

i = 2  # 3
j = 2  # 9
Ma = 'Ma'+str(i)
tUpDown = 'tU'+str(i) + 'D'+str(j)  # 交易信号 数组列
rUpDown = 'rU'+str(i) + 'D'+str(j)  # 收益率 数组列

cUp = 'cUp'+str(i)
cDown = 'cDown'+str(j)  # 死叉信号 数组列


####################################

# stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=SecurityID, period="daily", adjust="qfq")
# kLinedf = stock_zh_a_hist_df.tail(700).copy(deep=True)  # 140 280 420 560 700

kLinedf = kHistdf[kHistdf['代码'] == SecurityID].tail(700).copy(deep=True)  # 140 280 420 560 700
kLinedf = kLinedf.reset_index(drop=True)

eName_hist = eName_df[eName_df.dfcf.isin(kLinedf.columns.values.tolist())]
eName_hist = eName_hist[['dfcf', 'En']].reset_index(drop=True)
kLinedf.rename(columns=dict(eName_hist.values.tolist()), inplace=True)

for iList in [30, 60, 90, 120, 250, i , j]:
    kLinedf['Ma'+str(iList)] = talib.MA(kLinedf.Close.values, timeperiod=iList)  # 多均线

rowCount = 0
sumClose = 0.0
for rindex, row in kLinedf.iterrows():
    rowCount += 1
    sumClose += row['Close']
    maClose = round(sumClose/rowCount, 3)

    for iList in [30, 60, 90, 120, 250, i , j]:
        Ma = 'Ma'+str(iList)
        if math.isnan(row[Ma]):
            kLinedf.at[rindex, Ma] = maClose  # 移动均线 nan 补均值
        else:
            kLinedf.at[rindex, Ma] = round(row[Ma], 3)

kLinedf[cUp] = 0
kLinedf.loc[kLinedf.Close[(kLinedf.Close > kLinedf[Ma]) & (
    kLinedf.Close.shift() < kLinedf[Ma].shift())].index, cUp] = 1  # 金叉

kLinedf[cDown] = 0
kLinedf.loc[kLinedf.Close[(kLinedf.Close < kLinedf[Ma]) & (
    kLinedf.Close.shift() > kLinedf[Ma].shift())].index, cDown] = -1  # 死叉

kLinedf[tUpDown] = 0
for iList in [30, 60, 90, 120, 250]:
    kLinedf['On'+str(iList)] = 0
    kLinedf.loc[kLinedf.Close[(
        kLinedf.Close > kLinedf['Ma'+str(iList)])].index, 'On'+str(iList)] = 1  # 均线之上

kLinedf.loc[(kLinedf[cUp] == 1), tUpDown] = 1  # 交易信号 数组列 金叉+均线之上

# kLinedf.loc[(kLinedf[cUp] == 1) & ((kLinedf['On60'] == 1) | (kLinedf['On90'] == 1) | (
    # kLinedf['On120'] == 1) | (kLinedf['On250'] == 1)), tUpDown] = 1  # 交易信号 数组列 金叉+均线之上

kLinedf[rUpDown] = 1.0

for rindex, row in kLinedf.iterrows():  # 品种->均线1金叉->均线2死叉->行->循环
    signal = row[tUpDown]
    if posiDay > 0:  # 持仓
        kLinedf.at[rindex, rUpDown] = winRate
        posiDay += 1
        if row[tUpDown] == 1:  # 屏蔽持仓期的信号
            signal = 0
            kLinedf.at[rindex, tUpDown] = signal

        if row["High"] > buyPrice * (1 + winLim):  # 止盈平仓
            signal = -1
            
            sellPrice = round(buyPrice * (1 + winLim), 2)
            curRate = round(sellPrice / buyPrice * freeRate -1, 3)
            winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
            kLinedf.at[rindex, rUpDown] = winRate
            winCount += 1
            tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]

            posiDay = 0

        elif row["Low"] < buyPrice * (1 + lossLim): # 止损平仓
            signal = -2
            
            sellPrice = round(buyPrice * (1 + lossLim), 2)
            curRate = round(sellPrice / buyPrice * freeRate -1, 3)
            winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
            kLinedf.at[rindex, rUpDown] = winRate
            lossCount += 1
            tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]

            posiDay = 0

        elif row[cDown] == cDownLim:  # 死叉平仓
            signal = -4
            
            sellPrice = row["Close"]
            curRate = round(sellPrice / buyPrice * freeRate -1, 3)
            winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
            kLinedf.at[rindex, rUpDown] = winRate
            cDownCount += 1
            tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]
            posiDay = 0       

        elif posiDay >= dayLim:  # 超期平仓
            signal = -3
            
            sellPrice = row["Close"]
            curRate = round(sellPrice / buyPrice * freeRate -1, 3)
            winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
            kLinedf.at[rindex, rUpDown] = winRate
            OverCount += 1
            tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]
            posiDay = 0

        kLinedf.at[rindex, tUpDown] = signal

    elif posiDay == 0:
        if row[tUpDown] == 1:  # 交易信号，开仓 买入价
            signal = 1
            
            buyPrice = row["Close"]
            tradeCount += 1
            curRate = 0.0
            # print(rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], rUpDown, posiDay, signal, buyPrice, sellPrice, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount )

            tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]

            posiDay = 1
    else:
        pass

    kLinedf.at[rindex, rUpDown] = winRate

####################################

perWin = round(winCount/tradeCount *100, 1)  # 止盈平仓次数占比%
perLoss = round(lossCount/tradeCount *100, 1)  # 止盈平仓次数占比%
perOver = round(OverCount/tradeCount *100, 1)  # 超期平仓次数占比%
percDown = round(cDownCount/tradeCount *100, 1)  # 死叉平仓次数占比%

curTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print(f"[{curTime}] : {SecurityID} : {rUpDown} : {winRate}: 交易次数:{tradeCount}, {winLim}止盈:{winCount}次{perWin}%, {lossLim}止损:{lossCount}次{perLoss}%, {dayLim}日超期:{OverCount}次{perOver}%, {cDownLim}死叉:{cDownCount}次{percDown}%  ")

# maTacticdf.loc[maTacticdf.shape[0]] = [SecurityID, rUpDown, today, winRate, tradeCount,
        # winLim, winCount, perWin, lossLim, lossCount, perLoss, dayLim, OverCount,
        # perOver, cDownLim, cDownCount, percDown]

####################################


[2022-03-23 18:03:38] : 300390 : rU2D2 : 10.252: 交易次数:100, 0.233止盈:10次10.0%, -0.08止损:21次21.0%, 5日超期:68次68.0%, -2死叉:0次0.0%  


In [ ]:
##单股票 参数 批量 公共数据 kLinedf########

SecurityID = '300942' # 300390  300942

kLinedf = kHistdf[kHistdf['代码'] == SecurityID].tail(700).copy(deep=True)  # 140 280 420 560 700
kLinedf = kLinedf.reset_index(drop=True)

eName_hist = eName_df[eName_df.dfcf.isin(kLinedf.columns.values.tolist())]
eName_hist = eName_hist[['dfcf', 'En']].reset_index(drop=True)
kLinedf.rename(columns=dict(eName_hist.values.tolist()), inplace=True)

for iList in list(range(2, 20+1)) + [30, 60, 90, 120, 250]:
    kLinedf['Ma'+str(iList)] = talib.MA(kLinedf.Close.values, timeperiod=iList)  # 多均线

rowCount = 0
sumClose = 0.0
for rindex, row in kLinedf.iterrows():
    rowCount += 1
    sumClose += row['Close']
    maClose = round(sumClose/rowCount, 3)

    for iList in list(range(2, 20+1)) + [30, 60, 90, 120, 250]:
        Ma = 'Ma'+str(iList)
        if math.isnan(row[Ma]):
            kLinedf.at[rindex, Ma] = maClose  # 移动均线 nan 补均值
        else:
            kLinedf.at[rindex, Ma] = round(row[Ma], 3)

for i in range(2, 20+1):
    Ma = 'Ma'+str(i)
    cUp = 'cUp'+str(i)
    kLinedf[cUp] = 0
    kLinedf.loc[kLinedf.Close[(kLinedf.Close > kLinedf[Ma]) & (
        kLinedf.Close.shift() < kLinedf[Ma].shift())].index, cUp] = 1  # 金叉

    cDown = 'cDown'+str(i)
    kLinedf[cDown] = 0
    kLinedf.loc[kLinedf.Close[(kLinedf.Close < kLinedf[Ma]) & (
        kLinedf.Close.shift() > kLinedf[Ma].shift())].index, cDown] = -1  # 死叉

for iList in [30, 60, 90, 120, 250]:
    kLinedf['On'+str(iList)] = 0
    kLinedf.loc[kLinedf.Close[(
        kLinedf.Close > kLinedf['Ma'+str(iList)])].index, 'On'+str(iList)] = 1  # 均线之上


In [ ]:
##单股票 部分参数 批量循环 ########

print('start:', time.strftime("%Y-%m-%d %H:%M:%S"), '\n')

# SecurityID = '300390'
loopCount = 0
freeRate = 0.997  # 税费率

for i in [2]:  # 品种->均线1金叉->循环 range(2, 20+1)
    cUp = 'cUp'+str(i)
    for cDownLim in [-2]:  # 死叉-1信号 -2禁用 [-1, -2]
        for j in [2]:  # 品种->均线1金叉->均线2死叉->循环 range(2, 20+1)
            tUpDown = 'tU'+str(i) + 'D'+str(j)  # 交易信号 数组列
            rUpDown = 'rU'+str(i) + 'D'+str(j)  # 收益率 数组列
            cDown = 'cDown'+str(j)  # 死叉信号 数组列

            for winLim in [x/100 for x in range(3, 30, 2) ] + [0.10]:  # 止盈 [0.03, 0.05]  [x/100 for x in range(3, 30, 2) ] + [0.10]
                for lossLim in [-0.03, -0.05, -0.08]:  # 止损 -2禁用 [-0.03, -0.05, -0.08]
                    for dayLim in [3, 5, 7, 10]:  # 持仓天数限制 5

                        # SecurityID = '300390'
                        # winLim = 0.233  # 止盈 0.03, 0.05
                        # lossLim = -0.08  # 止损 -0.05 -0.03,-2
                        # dayLim = 5  # 持仓天数限制 3, 5, 7, 10
                        # cDownLim = -2  # 死叉信号  -1, -2 
                        # freeRate = 0.997  # 税费率

                        signal = 0    # 当前交易信号
                        winRate = 1.0  # 总收益率 *
                        curRate = 0.0  # 当前收益率 *
                        posiDay = 0   # 持仓天数
                        posiVol = 0   # 仓位 股数
                        initAmt = 0   # 初始金额
                        buyPrice = 0.0  # 买价
                        sellPrice = 0.0  # 卖价

                        tradeCount = 0  # 总交易次数 单品种 累计
                        winCount = 0  # 止盈平仓次数
                        lossCount = 0  # 止损平仓次数
                        OverCount = 0  # 超期平仓次数
                        cDownCount = 0  # 死叉平仓次数

                        # i = 2  # 3
                        # j = 2  # 9
                        # Ma = 'Ma'+str(i)
                        # tUpDown = 'tU'+str(i) + 'D'+str(j)  # 交易信号 数组列
                        # rUpDown = 'rU'+str(i) + 'D'+str(j)  # 收益率 数组列

                        # cUp = 'cUp'+str(i)
                        # cDown = 'cDown'+str(j)  # 死叉信号 数组列

                        ####################################

                        kLinedf[rUpDown] = 1.0
                        kLinedf[tUpDown] = 0
                        kLinedf.loc[(kLinedf[cUp] == 1), tUpDown] = 1  # 交易信号 数组列 金叉

                        # kLinedf.loc[(kLinedf[cUp] == 1) & ((kLinedf['On60'] == 1) | (kLinedf['On90'] == 1) | (
                            # kLinedf['On120'] == 1) | (kLinedf['On250'] == 1)), tUpDown] = 1  # 交易信号 数组列 金叉+均线之上

                        for rindex, row in kLinedf.iterrows():  # 品种->均线1金叉->均线2死叉->行->循环
                            signal = row[tUpDown]
                            if posiDay > 0:  # 持仓
                                kLinedf.at[rindex, rUpDown] = winRate
                                posiDay += 1
                                if row[tUpDown] == 1:  # 屏蔽持仓期的信号
                                    signal = 0
                                    kLinedf.at[rindex, tUpDown] = signal

                                if row["High"] > buyPrice * (1 + winLim):  # 止盈平仓
                                    signal = -1
                                    
                                    sellPrice = round(buyPrice * (1 + winLim), 2)
                                    curRate = round(sellPrice / buyPrice * freeRate -1, 3)
                                    winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
                                    kLinedf.at[rindex, rUpDown] = winRate
                                    winCount += 1
                                    tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]

                                    posiDay = 0

                                elif row["Low"] < buyPrice * (1 + lossLim): # 止损平仓
                                    signal = -2
                                    
                                    sellPrice = round(buyPrice * (1 + lossLim), 2)
                                    curRate = round(sellPrice / buyPrice * freeRate -1, 3)
                                    winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
                                    kLinedf.at[rindex, rUpDown] = winRate
                                    lossCount += 1
                                    tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]

                                    posiDay = 0

                                elif row[cDown] == cDownLim:  # 死叉平仓
                                    signal = -4
                                    
                                    sellPrice = row["Close"]
                                    curRate = round(sellPrice / buyPrice * freeRate -1, 3)
                                    winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
                                    kLinedf.at[rindex, rUpDown] = winRate
                                    cDownCount += 1
                                    tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]   
                                    posiDay = 0       

                                elif posiDay >= dayLim:  # 超期平仓
                                    signal = -3
                                    
                                    sellPrice = row["Close"]
                                    curRate = round(sellPrice / buyPrice * freeRate -1, 3)
                                    winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
                                    kLinedf.at[rindex, rUpDown] = winRate
                                    OverCount += 1
                                    tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]
                                    posiDay = 0

                                kLinedf.at[rindex, tUpDown] = signal

                            elif posiDay == 0:
                                if row[tUpDown] == 1:  # 交易信号，开仓 买入价
                                    signal = 1
                                    posiDay = 1
                                    buyPrice = row["Close"]
                                    tradeCount += 1
                                    curRate = 0.0
                                    # print(rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], rUpDown, posiDay, signal, buyPrice, sellPrice, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount )

                                    tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]

                            else:
                                pass

                            kLinedf.at[rindex, rUpDown] = winRate

                        ####################################

                        perWin = round(winCount/tradeCount *100, 1)  # 止盈平仓次数占比%
                        perLoss = round(lossCount/tradeCount *100, 1)  # 止盈平仓次数占比%
                        perOver = round(OverCount/tradeCount *100, 1)  # 超期平仓次数占比%
                        percDown = round(cDownCount/tradeCount *100, 1)  # 死叉平仓次数占比%

                        loopCount += 1
                        curTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
                        print(f"[{curTime}-{loopCount}-] : {SecurityID} : {rUpDown} : {winRate}: 交易次数:{tradeCount}, {winLim}止盈:{winCount}次{perWin}%, {lossLim}止损:{lossCount}次{perLoss}%, {dayLim}日超期:{OverCount}次{perOver}%, {cDownLim}死叉:{cDownCount}次{percDown}%  ")

                        maTacticdf.loc[maTacticdf.shape[0]] = [SecurityID, rUpDown, today, winRate, tradeCount,
                                winLim, winCount, perWin, lossLim, lossCount, perLoss, dayLim, OverCount,
                                perOver, cDownLim, cDownCount, percDown]

                        ####################################

            if cDownLim == -2:
                print(f'{i = }, {cDownLim = }, { j = }')
                break

maTacticdf.to_pickle(maTacticPath)
tradedf.to_pickle(tradePath)


In [ ]:
##单股票 全部参数 批量循环 ########

print('start:', time.strftime("%Y-%m-%d %H:%M:%S"), '\n')

# SecurityID = '300390'
loopCount = 0
freeRate = 0.997  # 税费率

for i in range(2, 20+1):  # 品种->均线1金叉->循环 range(2, 20+1)  [2]
    cUp = 'cUp'+str(i)
    for cDownLim in [-1, -2]:  # 死叉-1信号 -2禁用 [-1, -2] [-2]
        for j in range(2, 20+1):  # 品种->均线1金叉->均线2死叉->循环 range(2, 20+1)  [2]
            tUpDown = 'tU'+str(i) + 'D'+str(j)  # 交易信号 数组列
            rUpDown = 'rU'+str(i) + 'D'+str(j)  # 收益率 数组列
            cDown = 'cDown'+str(j)  # 死叉信号 数组列

            for winLim in [x/100 for x in range(3, 30, 2) ] + [0.10]:  # 止盈 [0.03, 0.05]  [x/100 for x in range(3, 30, 2) ] + [0.10]
                for lossLim in [-0.03, -0.05, -0.08, -2]:  # 止损 -2禁用 [-0.03, -0.05, -0.08]
                    for dayLim in [3, 5, 7, 10]:  # 持仓天数限制 5

                        # SecurityID = '300390'
                        # winLim = 0.233  # 止盈 0.03, 0.05
                        # lossLim = -0.08  # 止损 -0.05 -0.03,-2
                        # dayLim = 5  # 持仓天数限制 3, 5, 7, 10
                        # cDownLim = -2  # 死叉信号  -1, -2 
                        # freeRate = 0.997  # 税费率

                        signal = 0    # 当前交易信号
                        winRate = 1.0  # 总收益率 *
                        curRate = 0.0  # 当前收益率 *
                        posiDay = 0   # 持仓天数
                        posiVol = 0   # 仓位 股数
                        initAmt = 0   # 初始金额
                        buyPrice = 0.0  # 买价
                        sellPrice = 0.0  # 卖价

                        tradeCount = 0  # 总交易次数 单品种 累计
                        winCount = 0  # 止盈平仓次数
                        lossCount = 0  # 止损平仓次数
                        OverCount = 0  # 超期平仓次数
                        cDownCount = 0  # 死叉平仓次数

                        # i = 2  # 3
                        # j = 2  # 9
                        # Ma = 'Ma'+str(i)
                        # tUpDown = 'tU'+str(i) + 'D'+str(j)  # 交易信号 数组列
                        # rUpDown = 'rU'+str(i) + 'D'+str(j)  # 收益率 数组列

                        # cUp = 'cUp'+str(i)
                        # cDown = 'cDown'+str(j)  # 死叉信号 数组列

                        ####################################

                        kLinedf[rUpDown] = 1.0
                        kLinedf[tUpDown] = 0
                        kLinedf.loc[(kLinedf[cUp] == 1), tUpDown] = 1  # 交易信号 数组列 金叉

                        # kLinedf.loc[(kLinedf[cUp] == 1) & ((kLinedf['On60'] == 1) | (kLinedf['On90'] == 1) | (
                            # kLinedf['On120'] == 1) | (kLinedf['On250'] == 1)), tUpDown] = 1  # 交易信号 数组列 金叉+均线之上

                        for rindex, row in kLinedf.iterrows():  # 品种->均线1金叉->均线2死叉->行->循环
                            signal = row[tUpDown]
                            if posiDay > 0:  # 持仓
                                kLinedf.at[rindex, rUpDown] = winRate
                                posiDay += 1
                                if row[tUpDown] == 1:  # 屏蔽持仓期的信号
                                    signal = 0
                                    kLinedf.at[rindex, tUpDown] = signal

                                if row["High"] > buyPrice * (1 + winLim):  # 止盈平仓
                                    signal = -1
                                    
                                    sellPrice = round(buyPrice * (1 + winLim), 2)
                                    curRate = round(sellPrice / buyPrice * freeRate -1, 3)
                                    winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
                                    kLinedf.at[rindex, rUpDown] = winRate
                                    winCount += 1
                                    tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]

                                    posiDay = 0

                                elif row["Low"] < buyPrice * (1 + lossLim): # 止损平仓
                                    signal = -2
                                    
                                    sellPrice = round(buyPrice * (1 + lossLim), 2)
                                    curRate = round(sellPrice / buyPrice * freeRate -1, 3)
                                    winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
                                    kLinedf.at[rindex, rUpDown] = winRate
                                    lossCount += 1
                                    tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]

                                    posiDay = 0

                                elif row[cDown] == cDownLim:  # 死叉平仓
                                    signal = -4
                                    
                                    sellPrice = row["Close"]
                                    curRate = round(sellPrice / buyPrice * freeRate -1, 3)
                                    winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
                                    kLinedf.at[rindex, rUpDown] = winRate
                                    cDownCount += 1
                                    tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]   
                                    posiDay = 0       

                                elif posiDay >= dayLim:  # 超期平仓
                                    signal = -3
                                    
                                    sellPrice = row["Close"]
                                    curRate = round(sellPrice / buyPrice * freeRate -1, 3)
                                    winRate = round(winRate * sellPrice / buyPrice * freeRate, 3)
                                    kLinedf.at[rindex, rUpDown] = winRate
                                    OverCount += 1
                                    tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]
                                    posiDay = 0

                                kLinedf.at[rindex, tUpDown] = signal

                            elif posiDay == 0:
                                if row[tUpDown] == 1:  # 交易信号，开仓 买入价
                                    signal = 1
                                    posiDay = 1
                                    buyPrice = row["Close"]
                                    tradeCount += 1
                                    curRate = 0.0
                                    # print(rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], rUpDown, posiDay, signal, buyPrice, sellPrice, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount )

                                    tradedf.loc[tradedf.shape[0]] = [rindex, SecurityID, row["TradeDate"], row["Close"], row["High"], row["Low"], winLim, lossLim, dayLim, cDownLim, rUpDown, posiDay, signal, buyPrice, sellPrice, curRate, winRate, tradeCount, winCount, lossCount, OverCount, cDownCount]

                            else:
                                pass

                            kLinedf.at[rindex, rUpDown] = winRate

                        ####################################

                        perWin = round(winCount/tradeCount *100, 1)  # 止盈平仓次数占比%
                        perLoss = round(lossCount/tradeCount *100, 1)  # 止盈平仓次数占比%
                        perOver = round(OverCount/tradeCount *100, 1)  # 超期平仓次数占比%
                        percDown = round(cDownCount/tradeCount *100, 1)  # 死叉平仓次数占比%

                        loopCount += 1
                        curTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
                        print(f"[{curTime}-{loopCount}-] : {SecurityID} : {rUpDown} : {winRate}: 交易次数:{tradeCount}, {winLim}止盈:{winCount}次{perWin}%, {lossLim}止损:{lossCount}次{perLoss}%, {dayLim}日超期:{OverCount}次{perOver}%, {cDownLim}死叉:{cDownCount}次{percDown}%  ")

                        maTacticdf.loc[maTacticdf.shape[0]] = [SecurityID, rUpDown, today, winRate, tradeCount,
                                winLim, winCount, perWin, lossLim, lossCount, perLoss, dayLim, OverCount,
                                perOver, cDownLim, cDownCount, percDown]

                        ####################################

            if cDownLim == -2:
                print(f'{i = }, {cDownLim = }, { j = }')
                break

maTacticdf.to_pickle(maTacticPath)
tradedf.to_pickle(tradePath)


In [ ]:
##结果 分析####################

# kLinedf = kHistdf[kHistdf['代码'] == SecurityID].tail(700).copy(deep=True)
Tmp = maTacticdf[maTacticdf['SecurityID'] == SecurityID].copy(deep=True)

print(Tmp[Tmp['winRate'] > 1.8  ])
